In [1]:
# !pip install webdriver_manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import pymysql

In [2]:
one = []  #기업명
two = []  #근무지
three = [] #경력
four = [] #채용공고
five = [] #태그
six = [] #마감일
seven = [] #근무지(상세)

title = []
name = []
career = []
loc = []
tag = []
timeout = []
loc2 = []

In [3]:
driver = webdriver.Chrome("")

In [4]:
url = 'https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.recommend_order&years=0&selected=873&selected=669&selected=1024&selected=655&selected=899&locations=seoul.all'
driver.get(url)
current_window_handle = driver.current_window_handle

In [6]:
def page_down():
    prev_height = driver.execute_script('return document.body.scrollHeight')

    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(2)

        curr_height = driver.execute_script('return document.body.scrollHeight')
        if curr_height == prev_height:
            break
        prev_height = curr_height

In [5]:
driver.find_element(By.XPATH, value='//*[@id="__next"]/div[3]/div[2]/ul/li[1]').click()

In [6]:
element = driver.find_element(By.CLASS_NAME, value = 'JobHeader_JobHeader__DBrFJ')

In [7]:
element.text

'오엠티아이코리아∙서울 서초구∙신입\n[신입] ASP.NET 개발자'

In [8]:
one.append(element.text)

In [9]:
print(element.text.split('\n')[0].split('∙')[0])
print(element.text.split('\n')[0].split('∙')[1])
print(element.text.split('\n')[0].split('∙')[2])
print(element.text.split('\n')[1])

name.append(element.text.split('\n')[0].split('∙')[0])
loc.append(element.text.split('\n')[0].split('∙')[1])
career.append(element.text.split('\n')[0].split('∙')[2])
title.append(element.text.split('\n')[1])

오엠티아이코리아
서울 서초구
신입
[신입] ASP.NET 개발자


In [24]:
element = driver.find_element(By.CLASS_NAME, value = 'CompanyTags_CompanyTags__wpSCo')

In [25]:
element.text

'태그\n커피·스낵바\n편의시설\n연봉상위11~20%\n50명이하\n설립10년이상'

In [26]:
element.text.split('\n')

['태그', '커피·스낵바', '편의시설', '연봉상위11~20%', '50명이하', '설립10년이상']

In [27]:
two.append(element.text.split('\n')[1:])

In [15]:
tag.append(element.text.split('\n')[1:])

In [16]:
tag

[['커피·스낵바', '편의시설', '연봉상위11~20%', '50명이하', '설립10년이상']]

In [31]:
element = driver.find_element(By.CLASS_NAME, value = 'JobDueTime_JobDueTime__iKbEO')

In [18]:
element.text

'마감일\n2024.05.19'

In [19]:
element.text.split('\n')

['마감일', '2024.05.19']

In [34]:
element.text.split('\n')[1]

'2024.05.19'

In [37]:
three.append(element.text.split('\n')[1])

In [38]:
timeout.append(element.text.split('\n')[1])

In [40]:
element = driver.find_element(By.CLASS_NAME, value = 'JobWorkPlace_JobWorkPlace__5fs5x')

In [41]:
element.text

'근무지역\n서울특별시 서초구 서초대로27길 54, 1층'

In [42]:
element.text.split('\n')

['근무지역', '서울특별시 서초구 서초대로27길 54, 1층']

In [43]:
four.append(element.text.split('\n')[1])

In [44]:
loc2.append(element.text.split('\n')[1])

In [45]:
driver.back()

In [46]:
df = pd.DataFrame()

In [48]:
df['회사명'] = name
df['근무지'] = loc
df['경력'] = career
df['채용공고'] = title
df['태그'] = tag
df['마감일'] = timeout
df['근무지_상세'] = loc2


In [49]:
df

,회사명,근무지,경력,채용공고,태그,마감일,근무지_상세
0,오엠티아이코리아,서울 서초구,신입,[신입] ASP.NET 개발자,"[커피·스낵바, 편의시설, 연봉상위11~20%, 50명이하, 설립10년이상]",2024.05.19,"서울특별시 서초구 서초대로27길 54, 1층"


In [83]:
def craw():
    for j in range(1,5):
        time.sleep(3)
        if j == 1:
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'JobHeader_JobHeader__DBrFJ')
                name.append(element.text.split('\n')[0].split('∙')[0])
                loc.append(element.text.split('\n')[0].split('∙')[1])
                career.append(element.text.split('\n')[0].split('∙')[2])
                title.append(element.text.split('\n')[1])

                one.append(element.text.split('\n')[0].split('∙')[0])
                two.append(element.text.split('\n')[0].split('∙')[1])
                three.append(element.text.split('\n')[0].split('∙')[2])
                four.append(element.text.split('\n')[1])
            except:
                name.append('')
                loc.append('')
                career.append('')
                title.append('')
        elif j == 2:
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'CompanyTags_CompanyTags__wpSCo')
                two.append(element.text.split('\n')[1:])
                tag.append(element.text.split('\n')[1:])
            except:
                two.append('')
                tag.append('')
        elif j == 3:
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'JobDueTime_JobDueTime__iKbEO')
                three.append(element.text.split('\n')[1])
                timeout.append(element.text.split('\n')[1])
            except:
                three.append('')
                timeout.append('')
        else:
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'JobWorkPlace_JobWorkPlace__5fs5x')
                four.append(element.text.split('\n')[1])
                loc2.append(element.text.split('\n')[1])
            except:
                four.append('')
                loc2.append('')

In [87]:


page_down()

for i in tqdm(range(1,200)):
    time.sleep(5)
    try:
        driver.find_element(By.XPATH, value=f'//*[@id="__next"]/div[3]/div[2]/ul/li[{i}]').click()
        craw()
        time.sleep(1)
        driver.back()
    except:
        print(f'{i}개의 데이터 수집완료')
        break

100%|█████████████████████████████████████████████████████████████████████████████████| 4/4 [01:14<00:00, 18.69s/it]


In [90]:
print(len(name))
print(len(title))
print(len(loc))
print(len(loc2))
print(len(tag))
print(len(timeout))
print(len(career))


4
4
4
4
4
4
4


In [7]:
df = pd.DataFrame()

In [8]:
df['기업명'] = name
df['채용공고'] = title
df['위치'] = loc
df['위치상세'] = loc2
df['태그'] = tag
df['마감일'] = timeout
df['경력'] = career


In [9]:
df['경력']

Series([], Name: 경력, dtype: float64)